# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
# import libraries
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from io import StringIO
import pandas as pd
import numpy as np
import re
import pickle

import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
#df.head()
X = df['message'] 
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # step 1 clean text, remove punctuation and turn into lower cases
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    # tokenize 
    words = word_tokenize(text)
    # remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Stem word tokens
    stemmed = [PorterStemmer().stem(w) for w in words]
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report_to_df(report):
    report = re.sub(r" +", " ", report).replace("avg / total", "avg/total").replace("\n ", "\n")
    report_df = pd.read_csv(StringIO("Classes" + report), sep=' ', index_col=0)        
    return(report_df)

In [8]:
def eval_col_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        report = report_to_df(classification_report(np.array(Y_train)[:, i], Y_train_pred[:, i])).loc['avg/total']  
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        report[3] = accuracy
        metrics.append(report)
        #break
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Precision', 'Recall', 'F1','accuracy'])
      
    return metrics_df
    #return metrics

In [9]:
# Calculate evaluation metrics for training set
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

In [10]:
temp = eval_col_metrics(np.array(Y_train), Y_train_pred, col_names)
temp

,Precision,Recall,F1,accuracy
related,0.99,0.99,0.99,0.989167
request,0.99,0.99,0.99,0.986929
offer,1.00,1.00,1.00,0.998830
aid_related,0.99,0.99,0.99,0.985200
medical_help,0.99,0.99,0.99,0.988506
medical_products,0.99,0.99,0.99,0.992371
search_and_rescue,0.99,0.99,0.99,0.993795
security,1.00,1.00,1.00,0.995626
military,1.00,1.00,1.00,0.995372
child_alone,1.00,1.00,1.00,1.000000


In [11]:
# Calculate evaluation metrics for test set
Y_test_pred = pipeline.predict(X_test)
eval_test = eval_col_metrics(np.array(Y_test), Y_test_pred, col_names)
eval_test

,Precision,Recall,F1,accuracy
related,0.99,0.99,0.99,0.813244
request,0.99,0.99,0.99,0.882820
offer,1.00,1.00,1.00,0.996033
aid_related,0.99,0.99,0.99,0.750534
medical_help,0.99,0.99,0.99,0.918218
medical_products,0.99,0.99,0.99,0.949496
search_and_rescue,0.99,0.99,0.99,0.973909
security,1.00,1.00,1.00,0.982606
military,1.00,1.00,1.00,0.969332
child_alone,1.00,1.00,1.00,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 20]}

cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 3)

# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.25160207506866034, total= 2.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.23466585291425085, total= 2.4min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.24122673176685994, total= 2.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.2537381751602075, total= 2.0min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.23237717424473603, total= 2.1min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.25587427525175466, total= 2.1min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.2374122673176686, total= 2.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, 

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 85.3min finished


In [13]:
tuned_model.best_params_

{'clf__estimator__n_estimators': 20, 'tfidf__use_idf': True, 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# Calculate evaluation metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

eval_tuned = eval_col_metrics(np.array(Y_test), tuned_pred_test, col_names)

In [ ]:
eval_tuned

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# Try using SVM instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__learning_rate': [0.001, 0.01, 0.1]
              }

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, verbose = 3)

# Find best parameters
np.random.seed(81)
tuned_model2 = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__learning_rate=0.001, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__learning_rate=0.001, tfidf__use_idf=True, vect__min_df=5, score=0.18034787915776626, total= 2.5min
[CV] clf__estimator__learning_rate=0.001, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.001, tfidf__use_idf=True, vect__min_df=5, score=0.18629844369850473, total= 2.5min
[CV] clf__estimator__learning_rate=0.001, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.4min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.001, tfidf__use_idf=True, vect__min_df=5, score=0.17699115044247787, total= 2.6min
[CV] clf__estimator__learning_rate=0.01, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__learning_rate=0.01, tfidf__use_idf=True, vect__min_df=5, score=0.2007934086054318, total= 2.5min
[CV] clf__estimator__learning_rate=0.01, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__learning_rate=0.01, tfidf__use_idf=True, vect__min_df=5, score=0.20231919438510834, total= 2.5min
[CV] clf__estimator__learning_rate=0.01, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__learning_rate=0.01, tfidf__use_idf=True, vect__min_df=5, score=0.1853829722306988, total= 2.6min
[CV] clf__estimator__learning_rate=0.1, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__learning_rate=0.1, tfidf__use_idf=True, vect__min_df=5, score=0.21345743057674701, total= 2.6min
[CV] clf__estimator__learning_rate=0.1, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__est

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 33.4min finished


### 9. Export your model as a pickle file

In [20]:
# Pickle best model
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))
pickle.dump(tuned_model2, open('disaster_model2.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.